# Data Science Project 1
- the dataset come from public community
- this dataset is divided into 3 parts every part is about eCommerce data, we use three part to analyze this dataset.


## part 1
First we will process tmall_order_report, this is a order dataset.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('./data/tmall_order_report.csv')
# print(dataset.columns)
dataset.columns = ['orderId','order_amount','payment','address','order_date','payment_date','refund']
dataset.head(10)

,orderId,order_amount,payment,address,order_date,payment_date,refund
0,1,178.8,0.0,上海,2020-02-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2020-02-20 23:59:54,2020-02-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2020-02-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2020-02-20 23:58:34,2020-02-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2020-02-20 23:57:04,2020-02-20 23:57:11,64.8
5,6,327.7,148.9,浙江省,2020-02-20 23:56:39,2020-02-20 23:56:53,178.8
6,7,357.0,357.0,天津,2020-02-20 23:56:36,2020-02-20 23:56:40,0.0
7,8,53.0,53.0,浙江省,2020-02-20 23:56:12,2020-02-20 23:56:16,0.0
8,9,43.0,0.0,湖南省,2020-02-20 23:54:53,2020-02-20 23:55:04,43.0
9,10,421.0,421.0,北京,2020-02-20 23:54:28,2020-02-20 23:54:33,0.0


In [3]:
# explore the data
dataset.info() # 28010 entries in total, 7 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   orderId       28010 non-null  int64  
 1   order_amount  28010 non-null  float64
 2   payment       28010 non-null  float64
 3   address       28010 non-null  object 
 4   order_date    28010 non-null  object 
 5   payment_date  24087 non-null  object 
 6   refund        28010 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


In [4]:
# check missing values
dataset.isnull().sum() # payment_date has many missing values
# I think the missing values in payment_date are due to the fact that the order has not been paid yet

orderId            0
order_amount       0
payment            0
address            0
order_date         0
payment_date    3923
refund             0
dtype: int64

In [5]:
# check the unique values of address column
print(dataset['address'].unique())
# extract the city from the address
dataset['address'] = dataset['address'].str.replace('自治区|省|维吾尔|回族|壮族','',regex=True)
print(dataset['address'].unique())

['上海' '内蒙古自治区' '安徽省' '湖南省' '江苏省' '浙江省' '天津' '北京' '四川省' '贵州省' '辽宁省' '河南省'
 '广西壮族自治区' '广东省' '福建省' '海南省' '江西省' '甘肃省' '河北省' '黑龙江省' '云南省' '重庆' '山西省'
 '吉林省' '山东省' '陕西省' '湖北省' '青海省' '新疆维吾尔自治区' '宁夏回族自治区' '西藏自治区']
['上海' '内蒙古' '安徽' '湖南' '江苏' '浙江' '天津' '北京' '四川' '贵州' '辽宁' '河南' '广西' '广东'
 '福建' '海南' '江西' '甘肃' '河北' '黑龙江' '云南' '重庆' '山西' '吉林' '山东' '陕西' '湖北' '青海'
 '新疆' '宁夏' '西藏']


## analyze the data in total

In [6]:
result = {}

# paid and unpaid orders
result['total_order'] = dataset['orderId'].count()
result['total_order_paid'] = dataset[dataset['payment_date'].notnull()]['orderId'].count()
result['total_order_unpaid'] = dataset[dataset['payment_date'].isnull()]['orderId'].count()
result['refund_order'] = dataset[dataset['refund']>0]['orderId'].count()

# total amount of orders and total amount of refund
result['order_amount'] = dataset[dataset['payment_date'].notnull()]['order_amount'].sum()

result['total_refund_amount'] = dataset['refund'].sum()

result['income'] = dataset[dataset['payment_date'].notnull()]['order_amount'].sum()



In [7]:
print(result['order_amount']-result['total_refund_amount'])
result

1902487.1500000004


{'total_order': 28010,
 'total_order_paid': 24087,
 'total_order_unpaid': 3923,
 'refund_order': 5646,
 'order_amount': 2474823.0700000003,
 'total_refund_amount': 572335.9199999999,
 'income': 2474823.0700000003}

In [8]:
# use echarts to visualize the data
from pyecharts import options as opts
from pyecharts.charts import Map, Bar, Line
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.faker import Faker


table = Table()

headers = ['Total Orders', 'Total Paid Orders', 'Total Unpaid Orders', 'Total Refund Orders', 'Total Refund Amount', 'DealRate','refundRate']

rows = [[
    result['total_order'],
    f'{result["total_order_paid"]/10000:.2f} W',
    f'{result["total_order_unpaid"]}',
    f'{result["refund_order"]}',
    f'{result["total_refund_amount"]/10000:.2f} W',
    f'{result["total_order_paid"]/result["total_order"]:.2f}',# deal rate
    f'{result["refund_order"]/result["total_order_paid"]:.2f}' # refund rate
]]
table.add(headers, rows)

table.set_global_opts( # set the title of the table
    title_opts=ComponentTitleOpts(title="Total analysis")
)

table.render_notebook()
# we see that DealRate is 0.86 and refundRate is 0.23 

Total Orders,Total Paid Orders,Total Unpaid Orders,Total Refund Orders,Total Refund Amount,DealRate,refundRate
28010,2.41 W,3923,5646,57.23 W,0.86,0.23


In [9]:

# CurrentConfig.ONLINE_HOST = "https://cdn.pyecharts.org/assets/"
# Then we analyze the data by city
result2 = dataset[dataset['payment_date'].notnull()].groupby('address').agg({'orderId':'count'})
result21 = result2.to_dict()['orderId']
print([*result21.items()])
c = (
    Map()
    .add("Total Orders", [*result21.items()], "china", is_map_symbol_show=False)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Total Orders by City"),
        visualmap_opts=opts.VisualMapOpts(max_=1000),
    )
)

c.render_notebook()

[('上海', 3060), ('云南', 667), ('内蒙古', 176), ('北京', 1853), ('吉林', 336), ('四川', 1752), ('天津', 1031), ('宁夏', 40), ('安徽', 528), ('山东', 1484), ('山西', 395), ('广东', 2022), ('广西', 353), ('新疆', 43), ('江苏', 1845), ('江西', 331), ('河北', 885), ('河南', 792), ('浙江', 1822), ('海南', 156), ('湖北', 57), ('湖南', 935), ('甘肃', 132), ('福建', 425), ('西藏', 2), ('贵州', 286), ('辽宁', 1012), ('重庆', 896), ('陕西', 441), ('青海', 18), ('黑龙江', 312)]


In [10]:
# time analysis
dataset['order_date'] = pd.to_datetime(dataset['order_date'])
dataset['payment_date'] = pd.to_datetime(dataset['payment_date'])

result3 = dataset.groupby(dataset['order_date'].apply(lambda x:x.strftime('%Y-%m-%d'))).agg({'orderId':'count'}).to_dict()['orderId']
# print(list(result3.keys()))
# print([result3.keys()])
c = (
    Line()
    .add_xaxis(list(result3.keys()))
    .add_yaxis("Total Orders", list(result3.values()))
    .set_global_opts(title_opts=opts.TitleOpts(title="Total Orders by Date"))
)

c.render_notebook()
# after 2020-02-14 the number of orders increased significantly

In [11]:
# for one day we see the orders by hour
result4 = dataset[dataset['order_date'].notnull()].groupby(dataset['order_date'].apply(lambda x:x.strftime('%H'))).agg({'orderId':'count'}).to_dict()['orderId']


c = (
    Bar()
    .add_xaxis(list(result4.keys()))
    .add_yaxis("Total Orders", list(result4.values()))
    .set_global_opts(title_opts=opts.TitleOpts(title="Total Orders by Hour"))
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                     markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]))
)

c.render_notebook()
# we see that most orders are placed between 10:00 and 23:00,especially between at night

In [12]:
# the time betwwen order and payment
time_diff = dataset['payment_date']-dataset['order_date']
time_diff[time_diff.notnull()].apply(lambda x:x.seconds/60).mean()

7.7399046511949745

## part2
- beauty industry data analysis

In [13]:
dataset2 = pd.read_csv('./data/Beauty_sales_data.csv')
print(dataset2.columns)
dataset2.columns = ['update_time', 'id', 'title', 'price', 'sale_count', 'comment_count','name']
dataset2.head(10)

Index(['update_time', 'id', 'title', 'price', 'sale_count', 'comment_count',
       '店名'],
      dtype='object')


,update_time,id,title,price,sale_count,comment_count,name
0,2016/11/14,A18164178225,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,139.0,26719.0,2704.0,自然堂
1,2016/11/14,A18177105952,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,194.0,8122.0,1492.0,自然堂
2,2016/11/14,A18177226992,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,99.0,12668.0,589.0,自然堂
3,2016/11/14,A18178033846,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,38.0,25805.0,4287.0,自然堂
4,2016/11/14,A18178045259,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,139.0,5196.0,618.0,自然堂
5,2016/11/14,A18178129035,自然堂 雪域纯粹滋润洗颜霜 110g 补水保湿 洗面奶女 深层清洁,88.0,42858.0,8426.0,自然堂
6,2016/11/14,A18178206572,CHANDO/自然堂雪润皙白晶采霜（滋润型）50g 滋润修护面霜 正品,139.0,3027.0,304.0,自然堂
7,2016/11/14,A18190290933,自然堂 活泉深层净化控油凝露60g 控油补水保湿滋润 活泉精华正品,86.0,6925.0,740.0,自然堂
8,2016/11/14,A18250630014,CHANDO/自然堂 凝时鲜颜肌活修护精华液35ml 淡化细纹护肤精华,216.0,4649.0,630.0,自然堂
9,2016/11/14,A18422797881,CHANDO/自然堂活泉保湿精华套装 保湿补水滋润控油面部护理套装,258.0,2293.0,43.0,自然堂


In [14]:
dataset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27598 entries, 0 to 27597
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   update_time    27598 non-null  object 
 1   id             27598 non-null  object 
 2   title          27598 non-null  object 
 3   price          27598 non-null  float64
 4   sale_count     25244 non-null  float64
 5   comment_count  25244 non-null  float64
 6   name           27598 non-null  object 
dtypes: float64(3), object(4)
memory usage: 1.5+ MB


In [15]:
# check missing values
dataset2.isnull().sum() # sale_count and comment_count have missing values

update_time         0
id                  0
title               0
price               0
sale_count       2354
comment_count    2354
name                0
dtype: int64

In [16]:
# check the unique values of the dataset
print(dataset2[dataset2.duplicated()].count())
# 86 duplicated rows we need to remove them
dataset21 = dataset2.drop_duplicates()
dataset21.reset_index(drop=True,inplace=True)
dataset21.isnull().sum()

update_time      86
id               86
title            86
price            86
sale_count       82
comment_count    82
name             86
dtype: int64


update_time         0
id                  0
title               0
price               0
sale_count       2350
comment_count    2350
name                0
dtype: int64

In [17]:
# process the missing values, we use average value to fill the missing values
dataset21['sale_count'].fillna(dataset21['sale_count'].mean(),inplace=True)

dataset21['update_time'] = pd.to_datetime(dataset21['update_time']).apply(lambda x:x.strftime('%Y-%m-%d'))

dataset21[dataset21['sale_count']>0].sort_values(by='sale_count',ascending=False).head(10)

C:\Users\86156\AppData\Local\Temp\ipykernel_20912\3306400039.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset21['sale_count'].fillna(dataset21['sale_count'].mean(),inplace=True)
C:\Users\86156\AppData\Local\Temp\ipykernel_20912\3306400039.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset21['sale_count'].fillna(dataset21['s

,update_time,id,title,price,sale_count,comment_count,name
17026,2016-11-14,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,42.0,1923160.0,197949.0,妮维雅
17126,2016-11-13,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,37.9,1921582.0,198774.0,妮维雅
17228,2016-11-12,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,37.9,1920083.0,199062.0,妮维雅
17339,2016-11-11,A24304992630,2瓶更划算*妮维雅男士洗面奶控油祛痘印保湿去黑头去油洁面乳护肤,35.0,1886100.0,199532.0,妮维雅
17470,2016-11-10,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,42.0,1827562.0,200154.0,妮维雅
17621,2016-11-09,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,42.0,1827064.0,200752.0,妮维雅
17770,2016-11-08,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,42.0,1826527.0,201255.0,妮维雅
17903,2016-11-07,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,42.0,1825851.0,201741.0,妮维雅
18039,2016-11-06,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,42.0,1825075.0,202414.0,妮维雅
18189,2016-11-05,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,42.0,1824218.0,202930.0,妮维雅


In [18]:
# amount of sales by product
dataset21['total_sales'] = dataset21['price']*dataset21['sale_count']
dataset21[dataset21['sale_count']>0].sort_values(by='total_sales',ascending=False).head(10)
# the brand name "相宜草本" has the highest sales amount

C:\Users\86156\AppData\Local\Temp\ipykernel_20912\863548365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset21['total_sales'] = dataset21['price']*dataset21['sale_count']


,update_time,id,title,price,sale_count,comment_count,name,total_sales
10117,2016-11-12,A13956982789,相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,444337.0,9509.0,相宜本草,96865466.0
9940,2016-11-14,A13956982789,相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,444271.0,9355.0,相宜本草,96851078.0
10027,2016-11-13,A13956982789,相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,444263.0,9427.0,相宜本草,96849334.0
10306,2016-11-10,A13956982789,【双11来啦】相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,433604.0,9712.0,相宜本草,94525672.0
10439,2016-11-09,A13956982789,【双11来啦】相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,433603.0,9823.0,相宜本草,94525454.0
10604,2016-11-08,A13956982789,【双11来啦】相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,433599.0,9952.0,相宜本草,94524582.0
10769,2016-11-07,A13956982789,【双11来啦】相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,433598.0,10072.0,相宜本草,94524364.0
10935,2016-11-06,A13956982789,【双11来啦】相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,433595.0,10218.0,相宜本草,94523710.0
11100,2016-11-05,A13956982789,【双11来啦】相宜本草红石榴护肤化妆品套装女美白补水乳保湿,218.0,433591.0,10327.0,相宜本草,94522838.0
17026,2016-11-14,A24304992630,德国妮维雅男士洗面奶控油祛痘印保湿去黑头去油清洁面乳液护肤品,42.0,1923160.0,197949.0,妮维雅,80772720.0


### Data analysis and visualization

In [19]:
# amount of sale by day
result = dataset21.groupby('update_time').agg({'total_sales':'sum'}).to_dict()['total_sales']
c = (
    Line()
    .add_xaxis(list(result.keys()))
    .add_yaxis("Total Sales", list(result.values()))
    .set_global_opts(title_opts=opts.TitleOpts(title="Total Sales by Date"))
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                        markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average",name="average"),
                                                               opts.MarkLineItem(type_="max",name="max"),
                                                               opts.MarkLineItem(type_="min",name="min")]))
)
c.render_notebook()

In [20]:
# the best selling product
dts = list(dataset21['update_time'].unique())
dts.reverse()

In [21]:
from pyecharts import options as opts
from pyecharts.charts import Map, Timeline, Bar, Line, Pie
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts

tl = Timeline()

tl.add_schema(
    play_interval=1000,
    is_auto_play=True,
    is_loop_play=True,
)

for dt in dts:
    item = dataset21[dataset21['update_time'] <= dt].groupby('name').agg({'sale_count':'sum','total_sales':'sum'}).sort_values(by='sale_count',ascending=False).to_dict()
    # print(item)
    bar = (
        Bar()
        .add_xaxis([*item['sale_count'].keys()])
        .add_yaxis("Total Sales", [round(val/10000,2) for val in item['sale_count'].values()])
        .add_yaxis("Total Amount", [round(val/10000/1000,2) for val in item['total_sales'].values()])
        .reversal_axis()
        .set_global_opts(title_opts=opts.TitleOpts(title=f"Total Sales by Product on {dt}"))
    )

    tl.add(bar, dt)

tl.render_notebook()

In [22]:
item = dataset21.groupby('name').agg({'sale_count':'sum','total_sales':'sum'}).sort_values(by='sale_count',ascending=False).to_dict()['sale_count']

# print(item)
item = {k:round(v/10000,2) for k,v in item.items()}
# print(item)

c = (
    Pie()
    .add("sale_count", [list(z) for z in zip(item.keys(),item.values())])
    .set_global_opts(title_opts=opts.TitleOpts(title="Total Sales by Product"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)

c.render_notebook()

In [23]:
dataset21.columns

Index(['update_time', 'id', 'title', 'price', 'sale_count', 'comment_count',
       'name', 'total_sales'],
      dtype='object')

In [24]:
# the expensive product
item = dataset21.groupby('name').agg({'price':'mean'}).sort_values(by='price',ascending=False).to_dict()['price']

c = (
    Bar()
    .add_xaxis([*item.keys()])
    .add_yaxis("Price", [round(val,2) for val in item.values()])
    .reversal_axis()
    .set_global_opts(title_opts=opts.TitleOpts(title="Mean Price by Product"))
)

c.render_notebook() # the most expensive product is "娇兰"

## part3
- dataset:Daily_cosmetics.xlsx 
- sheet0: order data
- sheet1: commody infomation

In [113]:
dataset31 = pd.read_excel('./data/Daily_cosmetics.xlsx',sheet_name=0)

dataset32 = pd.read_excel('./data/Daily_cosmetics.xlsx',sheet_name=1)

### sheet0: order list

In [114]:
print(dataset31.columns)
dataset31.columns = ['orderId','date','customerId','area','province','city','commodyId','commodyAmount','commodyPrice','totalAmount']
dataset31.head(10)

Index(['订单编码', '订单日期', '客户编码', '所在区域', '所在省份', '所在地市', '商品编号', '订购数量', '订购单价',
       '金额'],
      dtype='object')


,orderId,date,customerId,area,province,city,commodyId,commodyAmount,commodyPrice,totalAmount
0,D31313,2019-05-16 00:00:00,S22796,东区,浙江省,台州市,X091,892,214,190888.0
1,D21329,2019-05-14 00:00:00,S11460,东区,安徽省,宿州市,X005,276,185,51060.0
2,D22372,2019-08-26 00:00:00,S11101,北区,山西省,忻州市,X078,1450,116,168200.0
3,D31078,2019-04-08 00:00:00,S10902,北区,吉林省,延边朝鲜族自治州,X025,1834,102,187068.0
4,D32470,2019-04-11 00:00:00,S18696,北区,北京市,北京市,X010,887,58,51446.0
5,D28336,2019-02-08 00:00:00,S17681,西区,云南省,红河哈尼族彝族自治州,X039,1102,108,119016.0
6,D29820,2019#3#11,S13780,南区,广东省,江门市,X011,1180,226,266680.0
7,D31449,2019-05-01 00:00:00,S12455,南区,广西壮族自治区,百色市,X051,1236,203,250908.0
8,D22542,2019-09-18 00:00:00,S10865,北区,内蒙古自治区,呼和浩特市,X089,1101,104,114504.0
9,D31859,2019-01-17 00:00:00,S17785,西区,云南省,曲靖市,X001,235,121,28435.0


In [115]:
dataset31.info() # 31452 entries in total, 10 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31452 entries, 0 to 31451
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   orderId        31452 non-null  object 
 1   date           31452 non-null  object 
 2   customerId     31452 non-null  object 
 3   area           31450 non-null  object 
 4   province       31450 non-null  object 
 5   city           31452 non-null  object 
 6   commodyId      31451 non-null  object 
 7   commodyAmount  31450 non-null  object 
 8   commodyPrice   31448 non-null  object 
 9   totalAmount    31448 non-null  float64
dtypes: float64(1), object(9)
memory usage: 2.4+ MB


In [116]:
print(dataset32.columns)
dataset32.columns = ['commodyId','commodyName','commodyType','commodyBrand','commodyPrice']
dataset32.head(10)

Index(['商品编号', '商品名称', '商品小类', '商品大类', '销售单价'], dtype='object')


,commodyId,commodyName,commodyType,commodyBrand,commodyPrice
0,X001,商品1,面膜,护肤品,121
1,X002,商品2,面膜,护肤品,141
2,X003,商品3,面膜,护肤品,168
3,X004,商品4,面膜,护肤品,211
4,X005,商品5,面膜,护肤品,185
5,X006,商品6,面膜,护肤品,159
6,X007,商品7,面膜,护肤品,213
7,X008,商品8,面膜,护肤品,119
8,X009,商品9,面膜,护肤品,92
9,X010,商品10,面膜,护肤品,58


In [117]:
dataset32.info() # 122 entries in total, 5 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   commodyId     122 non-null    object
 1   commodyName   122 non-null    object
 2   commodyType   122 non-null    object
 3   commodyBrand  122 non-null    object
 4   commodyPrice  122 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 4.9+ KB


In [118]:
# check missing values
print(dataset31.isnull().sum()) # area province and some other columns have missing values but little
# need clean the data
print('-----------------')
print(dataset32.isnull().sum())

orderId          0
date             0
customerId       0
area             2
province         2
city             0
commodyId        1
commodyAmount    2
commodyPrice     4
totalAmount      4
dtype: int64
-----------------
commodyId       0
commodyName     0
commodyType     0
commodyBrand    0
commodyPrice    0
dtype: int64


In [119]:
# check the unique values of the dataset
print(dataset31[dataset31.duplicated()].count()) # 6 duplicated rows
dataset31.drop_duplicates(inplace=True)
dataset31.reset_index(drop=True,inplace=True)
print(dataset31[dataset31.duplicated()].count())

orderId          6
date             6
customerId       6
area             6
province         6
city             6
commodyId        6
commodyAmount    6
commodyPrice     6
totalAmount      6
dtype: int64
orderId          0
date             0
customerId       0
area             0
province         0
city             0
commodyId        0
commodyAmount    0
commodyPrice     0
totalAmount      0
dtype: int64


In [120]:
dataset32[dataset32.duplicated()].count() # no duplicated rows

commodyId       0
commodyName     0
commodyType     0
commodyBrand    0
commodyPrice    0
dtype: int64

In [122]:
# fill the missing values --> order data
dataset31['area'].fillna(dataset31['area'].mode()[0],inplace=True)
dataset31['province'].fillna(dataset31['province'].mode()[0],inplace=True)
dataset31 = dataset31[dataset31['commodyId'].notnull()]
dataset31.reset_index(drop=True,inplace=True)
dataset31['commodyPrice'] = dataset31['commodyPrice'].astype(str).str.replace('元|个','', regex=True)
dataset31['commodyAmount'] = dataset31['commodyAmount'].astype(str).str.replace('元|个','', regex=True)
dataset31['commodyPrice'] = dataset31['commodyPrice'].astype(float)
dataset31['commodyAmount'] = dataset31['commodyAmount'].astype(float)
dataset31['commodyAmount'].fillna(dataset31['commodyAmount'].mean(),inplace=True)
dataset31['commodyPrice'].fillna(dataset31['commodyPrice'].mean(),inplace=True)
dataset31[dataset31['totalAmount'].isnull()]['totalAmount'] = dataset31['commodyAmount']*dataset31['commodyPrice']

dataset31.isnull().sum() # --> no missing values

C:\Users\86156\AppData\Local\Temp\ipykernel_20912\292970224.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset31['area'].fillna(dataset31['area'].mode()[0],inplace=True)
C:\Users\86156\AppData\Local\Temp\ipykernel_20912\292970224.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

orderId          0
date             0
customerId       0
area             0
province         0
city             0
commodyId        0
commodyAmount    0
commodyPrice     0
totalAmount      3
dtype: int64

In [127]:
dataset31['date'].astype(str)

0        2019-05-16 00:00:00
1        2019-05-14 00:00:00
2        2019-08-26 00:00:00
3        2019-04-08 00:00:00
4        2019-04-11 00:00:00
                ...         
31440    2019-05-03 00:00:00
31441               2019#3#3
31442    2019-09-27 00:00:00
31443    2019-08-06 00:00:00
31444    2019-07-12 00:00:00
Name: date, Length: 31445, dtype: object

In [130]:
dataset31['date'] = dataset31['date'].apply(lambda x: pd.to_datetime(x, format='%Y#%m#%d') if isinstance(x, str) else x)


In [132]:
dataset31['date'].sort_values(ascending=False).head(10) # we have a dirty data 2050

20797   2050-06-09
30856   2019-09-30
131     2019-09-30
18203   2019-09-30
29154   2019-09-30
14129   2019-09-30
27902   2019-09-30
14125   2019-09-30
29102   2019-09-30
29071   2019-09-30
Name: date, dtype: datetime64[ns]

In [ ]:
dataset31 = dataset31[dataset31['date']<'2021-01-01']

In [133]:
dataset31.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31445 entries, 0 to 31444
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   orderId        31445 non-null  object        
 1   date           31445 non-null  datetime64[ns]
 2   customerId     31445 non-null  object        
 3   area           31445 non-null  object        
 4   province       31445 non-null  object        
 5   city           31445 non-null  object        
 6   commodyId      31445 non-null  object        
 7   commodyAmount  31445 non-null  float64       
 8   commodyPrice   31445 non-null  float64       
 9   totalAmount    31442 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 2.4+ MB


In [138]:
dataset31[dataset31['totalAmount'].isnull()]

,orderId,date,customerId,area,province,city,commodyId,commodyAmount,commodyPrice,totalAmount
25029,D22086,2019-09-11,S22887,南区,江西省,九江市,X011,1698.000000,160.285732,NaN
25030,D23111,2019-04-22,S10909,东区,浙江省,温州市,X096,804.000000,160.285732,NaN
31444,D31856,2019-07-12,S17476,东区,湖北省,武汉市,X019,885.071778,160.285732,NaN


In [141]:
dataset31.dtypes

orderId                  object
date             datetime64[ns]
customerId               object
area                     object
province                 object
city                     object
commodyId                object
commodyAmount           float64
commodyPrice            float64
totalAmount             float64
dtype: object

In [142]:
dataset31['commodyAmount'] = pd.to_numeric(dataset31['commodyAmount'], errors='coerce')
dataset31['commodyPrice'] = pd.to_numeric(dataset31['commodyPrice'], errors='coerce')
dataset31.loc[dataset31['totalAmount'].isnull(), 'totalAmount'] = (
    dataset31['commodyAmount'] * dataset31['commodyPrice']
)

print(dataset31['totalAmount'].isnull().sum())

0


In [143]:
dataset31.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31445 entries, 0 to 31444
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   orderId        31445 non-null  object        
 1   date           31445 non-null  datetime64[ns]
 2   customerId     31445 non-null  object        
 3   area           31445 non-null  object        
 4   province       31445 non-null  object        
 5   city           31445 non-null  object        
 6   commodyId      31445 non-null  object        
 7   commodyAmount  31445 non-null  float64       
 8   commodyPrice   31445 non-null  float64       
 9   totalAmount    31445 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 2.4+ MB


In [146]:
print(dataset31['province'].unique())
dataset31['province'] = dataset31['province'].str.replace('省|市|自治区|维吾尔|回族|壮族','',regex=True)
print(dataset31['province'].unique())

['浙江省' '安徽省' '山西省' '吉林省' '北京市' '云南省' '广东省' '广西壮族自治区' '内蒙古自治区' '新疆维吾尔自治区'
 '湖北省' '江苏省' '甘肃省' '四川省' '河南省' '福建省' '陕西省' '辽宁省' '山东省' '江西省' '重庆市' '河北省'
 '湖南省' '上海市' '贵州省' '天津市' '海南省' '宁夏回族自治区' '黑龙江省']
['浙江' '安徽' '山西' '吉林' '北京' '云南' '广东' '广西' '内蒙古' '新疆' '湖北' '江苏' '甘肃' '四川'
 '河南' '福建' '陕西' '辽宁' '山东' '江西' '重庆' '河北' '湖南' '上海' '贵州' '天津' '海南' '宁夏'
 '黑龙江']


In [150]:
dataset31['customerId'] = dataset31['customerId'].astype(str).str.replace('编号','',regex=True)

### visualization

In [152]:
dataset31.columns

Index(['orderId', 'date', 'customerId', 'area', 'province', 'city',
       'commodyId', 'commodyAmount', 'commodyPrice', 'totalAmount', 'month'],
      dtype='object')

In [156]:
dataset31['month'] = dataset31['date'].dt.month
item = dataset31.groupby('month').agg({'commodyAmount':'sum','totalAmount':'sum'}).to_dict()
print(item)
c = (
    Bar()
    .add_xaxis([*item['commodyAmount'].keys()])
    .add_yaxis("Total Amount", [round(val/10000/100,2) for val in item['totalAmount'].values()])
    .add_yaxis("Total Sales", [round(val/10000,2) for val in item['commodyAmount'].values()])
    .set_global_opts(title_opts=opts.TitleOpts(title="Total Sales by Month"))
)   

c.render_notebook() # we see that the sales amount is the highest in July

{'commodyAmount': {1: 1947407.0, 2: 2148249.0, 3: 2568902.0, 4: 3185347.0, 5: 3590871.0, 6: 3554518.0, 7: 3887019.0717784, 8: 3833576.0, 9: 3115193.0}, 'totalAmount': {1: 313620183.0, 2: 343771307.0, 3: 413361291.0, 4: 505324866.72889763, 5: 575270541.0, 6: 574250043.0, 7: 622302427.3782803, 8: 607935472.0, 9: 498895549.17371666}}


In [165]:
# the sales amount by province
item = dataset31.groupby('province').agg({'commodyAmount':'sum','totalAmount':'sum'}).sort_values(by='totalAmount',ascending=False).to_dict()['commodyAmount']
print(item)
c = (
    Bar()
    .add_xaxis([*item.keys()])
    .add_yaxis("Total Amount (ten million)", [round(val/10000/100,2) for val in item.values()])
    .set_global_opts(title_opts=opts.TitleOpts(title="Total Sales by Province"))
)

c.render_notebook() # we see that the sales amount is the highest in 江苏

{'江苏': 4183480.0, '广东': 2952919.0, '浙江': 2358047.0, '福建': 1569114.0, '四川': 1351715.0, '安徽': 1353512.0, '湖南': 1314400.0, '山东': 1234058.0, '上海': 1125312.0, '河北': 964624.0, '江西': 934775.0, '广西': 914145.0, '湖北': 910423.0717783996, '重庆': 805434.0, '云南': 706882.0, '陕西': 647066.0, '辽宁': 564647.0, '北京': 561432.0, '河南': 552203.0, '山西': 516045.0, '贵州': 483007.0, '黑龙江': 481754.0, '吉林': 344333.0, '甘肃': 286306.0, '新疆': 202183.0, '天津': 155298.0, '海南': 134901.0, '内蒙古': 120813.0, '宁夏': 102254.0}


In [168]:
# sales by city
item = dataset31.groupby('city').agg({'commodyAmount':'sum','totalAmount':'sum'}).sort_values(by='totalAmount',ascending=False).head(20).to_dict()['commodyAmount']
print(item)
c = (
    Bar()
    .add_xaxis([*item.keys()])
    .add_yaxis("Total Amount (ten million)", [round(val/10000,2) for val in item.values()])
    .set_global_opts(title_opts=opts.TitleOpts(title="Total Sales by City"))
)

c.render_notebook() # we see that the sales amount is the highest in 南京

{'上海市': 1125312.0, '苏州市': 1010450.0, '重庆市': 805434.0, '深圳市': 778322.0, '泉州市': 738150.0, '北京市': 562749.0, '杭州市': 563136.0, '广州市': 524105.0, '南通市': 452168.0, '泰州市': 413528.0, '宁波市': 425098.0, '长沙市': 398747.0, '南京市': 395471.0, '无锡市': 381211.0, '常州市': 361425.0, '东莞市': 341821.0, '温州市': 343109.0, '扬州市': 314270.0, '福州市': 275812.0, '南昌市': 275555.0}


In [183]:
dataset32.columns = ['commodyId','commodyName','Type1','Type2','commodyPrice']
# the sales amount by product
dataset4 = pd.merge(dataset31,dataset32,on='commodyId',how='inner',suffixes=('_left', '_right'))
dataset4 = dataset4.drop(columns=[col for col in dataset4.columns if col.endswith('_right')])
dataset4.head(10)

,orderId,date,customerId,area,province,city,commodyId,commodyAmount,commodyPrice_left,totalAmount,month,commodyName,Type1,Type2
0,D31313,2019-05-16,S22796,东区,浙江,台州市,X091,892.0,214.0,190888.0,5,商品91,粉底,彩妆
1,D21329,2019-05-14,S11460,东区,安徽,宿州市,X005,276.0,185.0,51060.0,5,商品5,面膜,护肤品
2,D22372,2019-08-26,S11101,北区,山西,忻州市,X078,1450.0,116.0,168200.0,8,商品78,口红,彩妆
3,D31078,2019-04-08,S10902,北区,吉林,延边朝鲜族自治州,X025,1834.0,102.0,187068.0,4,商品25,眼霜,护肤品
4,D32470,2019-04-11,S18696,北区,北京,北京市,X010,887.0,58.0,51446.0,4,商品10,面膜,护肤品
5,D28336,2019-02-08,S17681,西区,云南,红河哈尼族彝族自治州,X039,1102.0,108.0,119016.0,2,商品39,面霜,护肤品
6,D29820,2019-03-11,S13780,南区,广东,江门市,X011,1180.0,226.0,266680.0,3,商品11,面膜,护肤品
7,D31449,2019-05-01,S12455,南区,广西,百色市,X051,1236.0,203.0,250908.0,5,商品51,爽肤水,护肤品
8,D22542,2019-09-18,S10865,北区,内蒙古,呼和浩特市,X089,1101.0,104.0,114504.0,9,商品89,粉底,彩妆
9,D31859,2019-01-17,S17785,西区,云南,曲靖市,X001,235.0,121.0,28435.0,1,商品1,面膜,护肤品


In [179]:
print(dataset31.columns)
print(dataset32.columns)


Index(['orderId', 'date', 'customerId', 'area', 'province', 'city',
       'commodyId', 'commodyAmount', 'commodyPrice', 'totalAmount', 'month'],
      dtype='object')
Index(['commodyId', 'commodyName', 'Type1', 'Type2', 'commodyPrice'], dtype='object')


In [186]:

dataset4.groupby(['Type1','Type2']).agg({'commodyAmount':'sum'}).sort_values(by=['Type2','commodyAmount'],ascending=[True, False]).head(10)

,,commodyAmount
Type1,Type2,
口红,彩妆,2.013024e+06
粉底,彩妆,1.188621e+06
睫毛膏,彩妆,5.873990e+05
眼影,彩妆,2.965990e+05
蜜粉,彩妆,4.553400e+04
面膜,护肤品,5.451914e+06
面霜,护肤品,4.566905e+06
爽肤水,护肤品,3.523687e+06
眼霜,护肤品,3.351057e+06


In [188]:
dataset31.columns

Index(['orderId', 'date', 'customerId', 'area', 'province', 'city',
       'commodyId', 'commodyAmount', 'commodyPrice', 'totalAmount', 'month'],
      dtype='object')

In [195]:
# use RFM model to analyze the data
"""
- Recency: How recently a customer has made a purchase
- Frequency: How often a customer makes a purchase
- Monetary Value: How much money a customer spends on purchases
"""

data_rfm = dataset31.groupby('customerId').agg({'date':max,'orderId':'count','totalAmount':'sum'}).reset_index()
data_rfm.columns = ['customerId','recency','frequency','monetary']

data_rfm['R'] = data_rfm['recency'].rank(pct=True,ascending=False)
data_rfm['F'] = data_rfm['frequency'].rank(pct=True)
data_rfm['M'] = data_rfm['monetary'].rank(pct=True)

data_rfm['R'] = np.where(data_rfm['R']>=0.5,1,0)
data_rfm['R'] = data_rfm['R'].astype(str)
data_rfm['F'] = np.where(data_rfm['F']>=0.5,1,0)
data_rfm['F'] = data_rfm['F'].astype(str)
data_rfm['M'] = np.where(data_rfm['M']>=0.5,1,0)
data_rfm['M'] = data_rfm['M'].astype(str)

data_rfm['RFM'] = data_rfm['R']+data_rfm['F']+data_rfm['M']

# we should divide the customers into 8 groups
rfm_mapping = {
    '111': 'Important Value Customer',
    '101': 'Important Growth Customer',
    '011': 'Important Retention Customer',
    '001': 'Important Recovery Customer',
    '110': 'General Value Customer',
    '100': 'General Growth Customer',
    '010': 'General Retention Customer',
    '000': 'General Recovery Customer'
}

data_rfm['RFM'] = data_rfm['RFM'].map(rfm_mapping)
data_rfm['RFM'].value_counts()

C:\Users\86156\AppData\Local\Temp\ipykernel_20912\46370495.py:8: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  data_rfm = dataset31.groupby('customerId').agg({'date':max,'orderId':'count','totalAmount':'sum'}).reset_index()


RFM
Important Retention Customer    334
General Growth Customer         327
Important Value Customer        184
General Recovery Customer       178
General Retention Customer       20
General Value Customer           16
Important Recovery Customer      14
Important Growth Customer        10
Name: count, dtype: int64

In [ ]:
"""
RFM analysis is a customer segmentation technique that uses past purchase behavior to divide customers into groups.
For Important Retention Customer: we should reward them with loyalty programs to retain them.
Important Value Customer :Maintain long-term relationships.
Important Recovery Customer :Issue large-amount coupons to attract users back.
Important Growth Customer :Guide users to apply for membership cards and points cards.

"""
# end of the analysis